In [ ]:
!pip install transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == 

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
import torch


In [ ]:
from datasets import load_dataset
dataset = load_dataset('cnn_dailymail', '3.0.0')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
dataset = load_dataset('cnn_dailymail', '3.0.0')
train_data = dataset['train'].select(range(200))
val_data = dataset['validation'].select(range(50))


In [ ]:
model_name = 'facebook/bart-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [ ]:
def preprocess(batch):
    inputs = tokenizer(batch['article'], max_length=512, truncation=True, padding='max_length')
    targets = tokenizer(batch['highlights'], max_length=128, truncation=True, padding='max_length')
    inputs['labels'] = targets['input_ids']
    return inputs

train_dataset = train_data.map(preprocess, batched=True, remove_columns=['article', 'highlights', 'id'])
val_dataset = val_data.map(preprocess, batched=True, remove_columns=['article', 'highlights', 'id'])


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir='./bart-summary-model',
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=2,
    logging_dir='./logs',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)
trainer.train()


<ipython-input-8-996812fb6854>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: thakredevyani0907 (thakredevyani0907-shri-ramdeobaba-college-of-engineering) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,No log,2.631284


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3353: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,2.631284
2,No log,1.868616


TrainOutput(global_step=200, training_loss=4.019791870117188, metrics={'train_runtime': 2534.1819, 'train_samples_per_second': 0.158, 'train_steps_per_second': 0.079, 'total_flos': 121947291648000.0, 'train_loss': 4.019791870117188, 'epoch': 2.0})

In [ ]:
model.save_pretrained('./bart-summary-model')
tokenizer.save_pretrained('./bart-summary-model')


('./bart-summary-model/tokenizer_config.json',
 './bart-summary-model/special_tokens_map.json',
 './bart-summary-model/vocab.json',
 './bart-summary-model/merges.txt',
 './bart-summary-model/added_tokens.json',
 './bart-summary-model/tokenizer.json')

In [ ]:
from transformers import pipeline
summarizer = pipeline('summarization', model='./bart-summary-model', tokenizer='./bart-summary-model')

text = """
NASA has announced the discovery of a new exoplanet that closely resembles Earth in size and temperature.
The planet, named Kepler-1649c, orbits in its star's habitable zone and may support liquid water.
This breakthrough was made possible using data collected by the retired Kepler space telescope...
"""

summary = summarizer(text, max_length=100, min_length=30, do_sample=False)
print("Summary:", summary[0]['summary_text'])


Device set to use cpu
Your max_length is set to 100, but your input_length is only 64. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=32)


Summary: NASA has announced the discovery of a new exoplanet that closely resembles Earth in size and temperature.The planet, named Kepler-1649c, orbits in its star's habitable zone and may support liquid water.This breakthrough was made possible using data collected by the retired Kepler space telescope...


In [ ]:
from transformers import pipeline
summarizer = pipeline('summarization', model='./bart-summary-model', tokenizer='./bart-summary-model')

text = """
Convolutional Neural Networks (CNNs) are a class of deep learning models designed for processing structured grid-like data, such as images. They are highly effective for computer vision tasks like image classification, object detection, and facial recognition.
1.	A CNN consists of multiple layers that transform an input image into a feature-rich representation. The key layers include:
2.	Convolutional Layer: This layer applies filters (kernels) to detect patterns such as edges, textures, and objects. The convolution operation extracts spatial features while preserving spatial relationships.
3.	Activation Function (ReLU): After convolution, the Rectified Linear Unit (ReLU) is applied to introduce non-linearity, ensuring the network can learn complex patterns.
4.	Pooling Layer: Pooling reduces the spatial dimensions of feature maps while retaining important features. Max-pooling is commonly used to extract the most prominent values, making the model translation-invariant.
5.	Fully Connected Layer: The high-level features extracted from convolutional and pooling layers are fed into a fully connected (dense) layer, which ultimately makes predictions.
6.	Softmax or Sigmoid Activation: These functions are used in the final layer to classify the input into different categories.

"""

summary = summarizer(text, max_length=100, min_length=30, do_sample=False)
print("Summary:", summary[0]['summary_text'])


Device set to use cpu


Summary: Convolutional Neural Networks (CNNs) are a class of deep learning models designed for processing structured grid-like data, such as images. They are highly effective for computer vision tasks like image classification, object detection, and facial recognition.Key features of CNNs include:1.TEXTUREA CNN consists of multiple layers that transform an input image into a feature-rich representation. The key layers include:TEXTURETEXTURETEXTURE coerc coerc coerc suscept suscept susceptable


In [ ]:
from IPython.core.display import display, HTML
from transformers import pipeline

# Load the summarizer from your fine-tuned model
summarizer = pipeline('summarization', model='./bart-summary-model', tokenizer='./bart-summary-model')

# Long article to summarize
text = """
Artificial Intelligence (AI) has rapidly become one of the most transformative technologies of the 21st century. It is revolutionizing industries ranging from healthcare and finance to entertainment and transportation. AI systems can now perform tasks that typically require human intelligence, such as recognizing speech, analyzing images, making decisions, and even generating creative content like music and art.

One of the most exciting applications of AI is in the field of healthcare. AI-powered tools are helping doctors diagnose diseases more accurately, discover new drugs faster, and provide personalized treatment recommendations based on a patient's unique genetic makeup. Meanwhile, self-driving cars, powered by AI, are changing the way we think about transportation by promising safer and more efficient travel.

However, the rise of AI also raises important ethical and social concerns. There are fears about job displacement due to automation, privacy issues from data collection, and potential misuse of AI in surveillance or warfare. As a result, many experts are calling for clear regulations and ethical frameworks to ensure that AI development remains safe, transparent, and aligned with human values.

Despite these challenges, AI continues to evolve at a rapid pace. As researchers and engineers push the boundaries of what machines can do, it is becoming increasingly important for society to stay informed and engaged in shaping the future of AI responsibly.
"""

# Generate summary
summary = summarizer(text, max_length=100, min_length=30, do_sample=False)[0]['summary_text']

# Display original text
display(HTML(f"<h4>📄 <u>Original Text</u>:</h4><div style='white-space: pre-wrap; font-size: 14px'>{text}</div>"))

# Display summary
display(HTML(f"<h4>📝 <u>Generated Summary</u>:</h4><div style='white-space: pre-wrap; color: green; font-size: 16px'><b>{summary}</b></div>"))


Device set to use cpu


In [ ]:
from IPython.core.display import display, HTML
from transformers import pipeline

# Load the summarizer from your fine-tuned model
summarizer = pipeline('summarization', model='./bart-summary-model', tokenizer='./bart-summary-model')

# Long article to summarize
text = """
Fingerprints are epidermal ridges and volar aspect arrangement on palms, fingers, and soles. The smaller crimson along with wrinkles on  skin over  the  palm  and  plantar aspects  is known  as  ridges. Fingerprint is an individuality principle with permanent features that cannot be replicated in an individual's entire life span. It is the most special, significant, and accurate characteristic in the human body [1, 2]. The  probability of two people having an identical pattern of fingerprints  is  one  in  64,000  million.  The  ridge pattern  of fingerprints  developed  during  the  fetal  period  do  not  change throughout  their  life  until  skin  decomposes.  The  pattern  of fingerprints  differ  widely  with  blood  groups  variation  [3,  4].  A fingerprint  is  a  representation  of  the  friction  ridge  on  all  parts. Epidermis  cornfield  layer  and  dermal  papillae  are  mainly associated with ridge patterns. During 3rd and 4th month of fetal life, primitive forms of epidermal ridges pattern could be characterized. Fingerprints  could  be  classified  as  loops,  arches,  whorl,  and composite.  The  palm  fingers  epidermal  ridges  can  be  fully developed during birth and remain unchanged for life except burns or  trauma  [5,  6].  Fingerprints, footprints,  iris scanning,  lip prints, and DNA profiling are all means of identifying person identity. The method  of examining  the impressions  of the  established friction skin  ridges is  known as  fingerprint  identification,  also known  as dactyloscopy. The palmar surface and digits  can be examined to see if the impressions are from the same finger [7].   The  presence or  absence of  inherited antigens  in  the red blood  cells of  human beings  is  classified  as  a blood  group. Rh method  and  ABO  are  two  significant  blood  groups.  Based  on plasma  antigen  existence,  ABO  blood  grouping  is  further categorized  into  A,  B,  AB,  and  O  variations  [8,  9].   Gene association with other characters complicates the genetics of blood groups.  The  higher  prevalence  of  duodenum  ulcer  in  “O”  blood group  population  and  stomach cancer  in  “A” blood  group people are  the  clinical  importance  of  varieties  in  blood  grouping.  In comparison to  the  general population,  the prevalence  of “O”  and “A”  groups  are significantly  higher  [10].  Various  Darmatoglyphics studies reported  a  significantly strong association between  blood groups and pattern of fingerprints [10-12]. Because of fingerprints implications  as  an  effective  means  of  identification,  this  study attempted to  evaluate finger  -print patterns and  their relationship with an individual blood group.
"""

# Generate summary
summary = summarizer(text, max_length=100, min_length=30, do_sample=False)[0]['summary_text']

# Display original text
display(HTML(f"<h4>📄 <u>Original Text</u>:</h4><div style='white-space: pre-wrap; font-size: 14px'>{text}</div>"))

# Display summary
display(HTML(f"<h4>📝 <u>Generated Summary</u>:</h4><div style='white-space: pre-wrap; color: green; font-size: 16px'><b>{summary}</b></div>"))

Device set to use cpu


In [ ]:
!zip -r bart-summary-model.zip bart-summary-model


  adding: bart-summary-model/ (stored 0%)
  adding: bart-summary-model/special_tokens_map.json (deflated 52%)
  adding: bart-summary-model/model.safetensors (deflated 8%)
  adding: bart-summary-model/checkpoint-200/ (stored 0%)
  adding: bart-summary-model/checkpoint-200/special_tokens_map.json (deflated 52%)
  adding: bart-summary-model/checkpoint-200/model.safetensors (deflated 8%)
  adding: bart-summary-model/checkpoint-200/tokenizer.json (deflated 82%)
  adding: bart-summary-model/checkpoint-200/config.json (deflated 64%)
  adding: bart-summary-model/checkpoint-200/merges.txt (deflated 53%)
  adding: bart-summary-model/checkpoint-200/rng_state.pth (deflated 24%)
  adding: bart-summary-model/checkpoint-200/optimizer.pt (deflated 9%)
  adding: bart-summary-model/checkpoint-200/tokenizer_config.json (deflated 75%)
  adding: bart-summary-model/checkpoint-200/generation_config.json (deflated 47%)
  adding: bart-summary-model/checkpoint-200/scheduler.pt (deflated 56%)
  adding: bart-summ

In [ ]:
!zip -r /content/bart-summary-model.zip bart-summary-model


updating: bart-summary-model/ (stored 0%)
updating: bart-summary-model/special_tokens_map.json (deflated 52%)
updating: bart-summary-model/model.safetensors (deflated 8%)
updating: bart-summary-model/checkpoint-200/ (stored 0%)
updating: bart-summary-model/checkpoint-200/special_tokens_map.json (deflated 52%)
updating: bart-summary-model/checkpoint-200/model.safetensors (deflated 8%)
updating: bart-summary-model/checkpoint-200/tokenizer.json (deflated 82%)
updating: bart-summary-model/checkpoint-200/config.json (deflated 64%)
updating: bart-summary-model/checkpoint-200/merges.txt (deflated 53%)
updating: bart-summary-model/checkpoint-200/rng_state.pth (deflated 24%)
updating: bart-summary-model/checkpoint-200/optimizer.pt (deflated 9%)
updating: bart-summary-model/checkpoint-200/tokenizer_config.json (deflated 75%)
updating: bart-summary-model/checkpoint-200/generation_config.json (deflated 47%)
updating: bart-summary-model/checkpoint-200/scheduler.pt (deflated 56%)
updating: bart-summ

In [ ]:
!cp /content/bart-summary-model.zip /content/drive/MyDrive/


cp: cannot create regular file '/content/drive/MyDrive/': No such file or directory
